In [1]:
import pandas as pd
import iotbx.pdb
import sys
import numpy as np
import pickle
import math
import matplotlib.pylab as plt
import time
import scipy as sp
import cctbx
import seaborn as sns
from scipy.optimize import fmin, fmin_cg, leastsq, fmin_l_bfgs_b
from scipy.interpolate import interp1d
import scipy.stats 
sns.set_palette("colorblind")
import requests
import re

In [2]:
df = pd.DataFrame()
def read_data(struct):
    rows=[]
    st = iotbx.pdb.input(file_name=struct)
    for atm in st.atoms_with_labels():
        if not atm.element==" H":
            rows.append({ 'b':atm.b,
                          'x': atm.xyz[0],
                          'y': atm.xyz[1],
                          'z': atm.xyz[2],
                          'chain': atm.chain_id,
                          'resname': atm.resname,
                          'Residue number': int(atm.resseq),
                          'combo': atm.icode+atm.resseq+atm.altloc,
                          'altloc': atm.altloc,
                          'Residue_label': atm.icode+atm.resseq+atm.altloc+' '+atm.resname,
                          'Residue_label2': atm.icode+atm.resseq+' '+atm.resname,
                          'atmname': atm.name })
    return pd.DataFrame(rows)

In [3]:
def get_uniprot_entries(pdb_id):
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    response = requests.get(url)
    pdb_data = response.text
    
    uniprot_entries = []
    
    for line in pdb_data.split('\n'):
        if line.startswith('DBREF') and 'UNP' in line:
            match = re.search(r'UNP\s+(\w+)', line)
            if match:
                uniprot_entry = match.group(1)
                chain_id = line[12]
                uniprot_entries.append((chain_id, uniprot_entry))
    
    return uniprot_entries

# Example usage
pdb_id = '6YIE'  # Replace with your desired PDB ID
uniprot_entries = get_uniprot_entries(pdb_id)
chdict={}
entries=[]
# Print the retrieved chains and UniProt entries
for chain_id, uniprot_entry in uniprot_entries:
    print(f"Chain {chain_id}: UniProt Entry {uniprot_entry}")
    chdict[chain_id]=uniprot_entry
    entries.append(uniprot_entry)
entries= list(set(entries))
entries


Chain A: UniProt Entry O15392
Chain B: UniProt Entry Q53HL2
Chain C: UniProt Entry Q9NQS7
Chain D: UniProt Entry O15392
Chain E: UniProt Entry Q53HL2
Chain F: UniProt Entry Q9NQS7


['Q53HL2', 'O15392', 'Q9NQS7']

In [4]:
df=read_data("6yie.pdb")


In [5]:
for key in chdict:
    df.loc[df.chain==key,'Entry']=chdict[key]

In [58]:
df

,b,x,y,z,chain,resname,Residue number,combo,altloc,Residue_label,Residue_label2,atmname,Entry
0,65.85,306.357,263.070,229.707,A,GLY,41,41,,41 GLY,41 GLY,N,Q6PI79
1,65.85,305.972,264.310,230.355,A,GLY,41,41,,41 GLY,41 GLY,CA,Q6PI79
2,65.85,304.529,264.407,230.816,A,GLY,41,41,,41 GLY,41 GLY,C,Q6PI79
3,65.85,304.056,265.486,231.132,A,GLY,41,41,,41 GLY,41 GLY,O,Q6PI79
4,65.31,303.832,263.272,230.861,A,THR,42,42,,42 THR,42 THR,N,Q6PI79
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34278,122.95,285.401,251.130,254.879,Y,DA,136,136,,136 DA,136 DA,N1,NaN
34279,122.95,286.546,251.079,255.566,Y,DA,136,136,,136 DA,136 DA,C2,NaN
34280,122.95,287.454,252.037,255.754,Y,DA,136,136,,136 DA,136 DA,N3,NaN
34281,122.95,287.085,253.169,255.136,Y,DA,136,136,,136 DA,136 DA,C4,NaN


In [6]:
tfs=[]
for entry in entries:
    _tf=pd.read_excel(entry+'_robbind.xlsx')
    tfs.append(_tf)
tf=pd.concat(tfs)
tf['Residue number']=tf['Position_fasta']
tf

,Unnamed: 0,Position_fasta,Entry,robbind,resname_fasta,Residue number
0,0,1,Q53HL2,0.2,M,1
1,1,2,Q53HL2,0.4,A,2
2,2,3,Q53HL2,0.5,P,3
3,3,4,Q53HL2,0.0,R,4
4,4,5,Q53HL2,0.0,K,5
...,...,...,...,...,...,...
910,910,911,Q9NQS7,0.0,L,911
911,911,912,Q9NQS7,0.0,A,912
912,912,913,Q9NQS7,0.0,Y,913
913,913,914,Q9NQS7,0.0,S,914


In [7]:
tdf=df[['Entry','chain','Residue number','resname']].drop_duplicates()
cdf=pd.merge(tdf, tf, on=['Entry', 'Residue number'])
cdf=cdf.sort_values(['chain','Residue number'])
cdf.to_excel('cdf_to_check.xlsx')

In [9]:
cdf=pd.read_excel('cdf_checked.xlsx')
mf = pd.merge(df, cdf, on=['Entry', 'Residue number'])
mf['chain']=mf['chain_x']
mf.to_excel('mf_to_check.xlsx')

In [10]:
mf

,b,x,y,z,chain_x,resname_x,Residue number,combo,altloc,Residue_label,...,atmname,Entry,Unnamed: 0,chain_y,resname_y,Unnamed: 0.1,Position_fasta,robbind,resname_fasta,chain
0,77.39,-5.141,0.441,-0.614,A,THR,5,5,,5 THR,...,N,O15392,0,A,THR,4,5,0.0,T,A
1,77.39,-5.141,0.441,-0.614,A,THR,5,5,,5 THR,...,N,O15392,1,D,THR,4,5,0.0,T,A
2,78.02,-4.945,1.739,-1.253,A,THR,5,5,,5 THR,...,CA,O15392,0,A,THR,4,5,0.0,T,A
3,78.02,-4.945,1.739,-1.253,A,THR,5,5,,5 THR,...,CA,O15392,1,D,THR,4,5,0.0,T,A
4,76.32,-6.111,2.000,-2.201,A,THR,5,5,,5 THR,...,C,O15392,0,A,THR,4,5,0.0,T,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7100,57.56,-20.190,46.033,-76.416,F,PRO,6,6,,6 PRO,...,C,Q9NQS7,472,F,PRO,5,6,0.0,P,F
7101,58.94,-19.067,46.329,-76.836,F,PRO,6,6,,6 PRO,...,O,Q9NQS7,472,F,PRO,5,6,0.0,P,F
7102,64.88,-21.527,47.886,-75.350,F,PRO,6,6,,6 PRO,...,CB,Q9NQS7,472,F,PRO,5,6,0.0,P,F
7103,67.64,-21.228,48.736,-74.181,F,PRO,6,6,,6 PRO,...,CG,Q9NQS7,472,F,PRO,5,6,0.0,P,F


In [11]:
rows=[]
st = iotbx.pdb.input(file_name="6yie.pdb")
for atm in st.atoms_with_labels():
    try:
        if (float(mf[(mf['Residue number']==int(atm.resseq))&(mf['chain']==atm.chain_id)]['robbind'].values[0]))>0:
            atm.set_b(float(mf[(mf['Residue number']==int(atm.resseq))&(mf['chain']==atm.chain_id)]['robbind'].values[0])*100)
            print (float(mf[(mf['Residue number']==int(atm.resseq))&(mf['chain']==atm.chain_id)]['robbind'].values[0])*100)
            #if (mf[(mf['Residue number']==int(atm.resseq))&(mf['chain']==atm.chain_id)]['resname'].values[0])!=atm.resname:
            #    print ('Something is fishy!')
        else:
            atm.set_b(0)
    except:
        atm.set_b(0)


6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
6.666666666666667
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
1.666666666666667
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
11.66666666666667
46.666666666666664
46.666666666666

100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.3333333333333

71.66666666666667
71.66666666666667
71.66666666666667
71.66666666666667
71.66666666666667
71.66666666666667
71.66666666666667
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
40.0
40.0
40.0
40.0
40.0
40.0
40.0
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0
40.0


50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
50.0
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
71.66666666666667
71.66666666666667
71.66666666666667
71.66666666666667
71.66666666666667
71.66666666666667
71.66666666666667
31.66666666666667
31.66666666666667
31.66666666666667
31.66666666666667
31.66666666666667


66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
65.0
65.0
65.0
65.0
65.0
65.0
65.0
65.0
65.0
65.0
65.0
65.0
65.0
65.0
65.0
65.0
65.0
65.0
65.0
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
53.333333333333336
53.333333333333336
53.333333333333336
53.333333333333336
53.333333333333336
53.333333333333336

45.0
45.0
45.0
45.0
45.0
45.0
45.0
45.0
45.0
45.0
45.0
45.0
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
8.333333333333332
36.66666666666666
36.66666666666666
36.66666666666666
36.66666666666666
36.66666666666666
36.66666666666666
36.66666666666666
51.66666666666667
51.66666666666667
51.66666666666667
51.66666666666667
51.66666666666667
51.66666666666667
51.66666666666667
51.66666666666667
51.66666666666667
51.66666666666667
51.66666666666667
51.66666666666667
51.66666666666667
51.66666666666667
51.66666666666667
51.66666666666667
51.66666666666667
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
38.33333333333334
8.333333333333332
8.333333333333332
8.333333333333332
8.33

41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
41.66666666666667
81.66666666666667
81.66666666666667
81.66666666666667
81.66666666666667
81.66666666666667
81.66666666666667
81.66666666666667
81.66666666666667
81.66666666666667
81.66666666666667
81.66666666666667
81.66666666666667
81.66666666666667
81.66666666666667
81.66666666666667
81.66666666666667
81.66666666666667
81.66666666666667
81.66666666666667
96.66666666666667
96.66666666666667
96.66666666666667
96.6666666

100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
100.0
100.0
100.0
100.0
100.0
100.0
100.0
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
66.66666666666666
88.33333333333333
88.33333333333333
88.33333333333333
88.33333333333333
88.33333333333333
88.33333333333333
88.33333333333333
88.33333333333333
88.33333333333333
88.3

100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33

In [12]:
st.write_pdb_file("mapped_6yie_everything.pdb")